# SpecAugment

Spectrogram Augmentation Paper

operates on the log mel spectrogram of the input audio, rather than the raw audio itself.

it directly acts on the log mel spectrogram as if it were an image, and does not require any additional data.

SpecAugment consists of three kinds of deformations of the log mel spectrogram. "The first is time warping, a deformation of the time-series in the time direction." The other two augmentations, inspired by "Cutout", proposed in computer vision[T. DeVries and G. Taylor "Improved Regularization of Convolutional Neural Networks with Cutout" in arXiv, 2017], are time and frequency masking, where we mask a block of consecutive time stops or mel frequency channels.

---------------------------------

## Time Warping 

-------------

Time Warping is applied via the function 'sparse_image_warp' of 'tensorflow'. Given a log mel spectrogram with $\tau$ time steps. A random point along the time axis (W, t - W) is to be warped either to the left or right by a distance 'w' chosen from a uniform distribution from 0 to the time warp parameter W along that line.

W : random point    
t : length    
w : moving distance   

## Frequency Masking
---------------------------------
Frequency masking is applied so that f consecutive mel frequency channels (f0, f0 + f) are masked, where f is first chosen from a uniform distribution from 0 to the frequency mask parameter F, and f0 is chosen from (0, v - f). v is the number of mel frequency channels.

f0 : random frequency point   
f  : masking length    
v  : end-point of frequency    
F  : Parameter

## Time Masking 
----------------------------------
Time masking is applied so that t consecutive time steps (t0, t0 + t) are masked, where t is first chosen from a uniform distribution form 0 to the time mask parameter T, and t0 is chosen from (0, $\tau$ - t).

t0 : random time point   
t : masking length   
$\tau$ : endpoint of time   
T : Parameter